
# Day 1: Introduction

We talked today about the massive transformation that electricity markets are witnessing, with the rapid growth of renewable power and explicity goal of fully decarbonizing the electricity market in coming years.

In this practice session, we will examine time series data from the Spanish electricity market, which has substantial intermittent renewable power (wind and solar). 

The data have been collected from publicly available sources (Red Eléctrica de España and OMIE, among others). The data are from the paper "Measuring the Impact of Wind Power in the Spanish Electricity Market," by Claire Petersen, Mar Reguant, and Lola Segura.

We need to load packages in Julia, similar to the import function in Python or the library functionality in R.

In [ ]:
using Pkg
# Pkg.add(["Binscatters", "CSV", "DataFrames", "Statistics", "Plots", "FixedEffectModels", "RegressionTables"])


To **load the libraries**, we use the command `using`. 

Here we will be loading a bunch of libraries so that we can load and use the data (`DataFrames`, `CSV`), compute statistics and manipulate data (`Statistics`) and make some nice plots (`Plots`, `Binscatters`). We will also be running some fixed-effects regressions (`FixedEffectModels`).

In [ ]:
using DataFrames
using CSV
using Binscatters
using Statistics
using Plots
using FixedEffectModels
using RegressionTables



We load the data using the CSV syntax (`CSV.read`) into a data frame called `df`. Make sure the data is in the same directory as the notebook or specify the full path name.

In [ ]:
cd("G:/.shortcut-targets-by-id/1LEj0H_g-jhr7WCtNwaDLaQ4EjziKUqfS/24EN01 - Empirical Methods for the Analysis of the Energy Transition/Practicals/day_1")
df = CSV.read("data_spain.csv", DataFrame)
df

## Summary Statistics

We start by displaying some statistics and plot hourly and yearly patterns of wind production and electricity demand. eltype determines the type of the variable. Note that Julia differentiates between Integers (1) and Floating-Point (1.0)


In [ ]:
describe(df)

In order to plot hourly and yearly patterns, we first need to combine the data at those levels. For that, we first define the groups for which the functions will be applied using `groupby`. `combine` is then used to compute the specified summary statistic. Finally, we rename the variable as `wind_mean`.

In [ ]:
#compute the mean for each hour and year
df_mean = combine(groupby(df, ["hour", "year"]), :wind => mean => :wind_mean, 
    :demand => mean => :demand_mean);

plot(df_mean.hour, df_mean.wind_mean, group = df_mean.year,
    seriestype = :line, linewidth = 3,
    title = "Wind blows more at night...",
    xlabel = "hour",
    ylabel = "Wind production",
    legend = :outerright)

In [ ]:
plot(df_mean.hour, df_mean.demand_mean, group = df_mean.year,
    seriestype = :line, linewidth = 3,
    title = "...and it is weakly correlated with demand",
    xlabel = "hour",
    ylabel = "Electricity Demand",
    legend = :outerright)

## The impacts of wind: a visual exploration

We will be plotting the **impacts of wind** on several outcomes of interest:
* Emissions
* Wholesale prices
* System costs
* Wholesale prices + system costs

We will be using the library `Binscatters` for plotting.

In [ ]:
scatter( df.wind_forecast, df.emis_tCO2)

In [ ]:
binscatter(df, @formula(emis_tCO2 ~ wind_forecast), 10, 
		seriestype = :scatterpath,
		title = "Wind reduces emissions",
		xlabel = "Wind forecast (GWh)",
		ylabel = "Hourly emissions (tons CO2)", 
		label="wind forecast",
		legend=:topright)
#we can add new specifications
binscatter!(df, @formula(emis_tCO2 ~ wind), 10,seriestype = :scatterpath,
		label="wind production")

In [ ]:
#we can add controls
binscatter(df, @formula(wholesale_price ~ wind_forecast), 10, 
		seriestype = :scatterpath,
		title = "Wind reduces wholesale prices",
		xlabel = "Wind forecast (GWh)",
		ylabel = "Wholesale price (EUR/MWh)",
		label = "no controls")

#   binscatter!(df, @formula(wholesale_price ~ wind_forecast + demand_forecast + fe(year) + fe(month) + fe(hour)), 10,seriestype = :scatterpath,
#   label = "controls")

In [ ]:
binscatter(df, @formula(system_costs ~ wind_forecast), 10, 
		seriestype = :scatterpath,
		title = "Wind increases system costs",
		xlabel = "Wind forecast (GWh)",
		ylabel = "System costs (EUR/MWh)")

In [ ]:
df.total_price = df.wholesale_price + df.system_costs
binscatter(df, @formula(total_price ~ wind_forecast), 10, 
	seriestype = :scatterpath,
	title = "But still reduces overall costs",
	xlabel = "Wind forecast (GWh)",
	ylabel = "Wholesale price + system costs (EUR/MWh)")

## Wind endogeneity

One can estimate the effects of wind using a regression framework. However, it is important to keep in mind that wind production can be endogenous.

In moments of very high forecasted wind, it is often the case that wind is discarded. This can create an endogeneity problem.

In [ ]:
scatter(df.wind_forecast, df.wind, xlabel="Forecasted wind", ylabel="Wind", legend=false, title="Discarded wind")

We can examine the endogeneity problem in the context of assessing the impact of wind on reliability and other congestion costs ("system costs").

On days of very high wind, measured wind production could be lower than expected, leading to a downward bias in our estimates: a difficult day with lots of wind appears as a day with low levels of wind in the data.

To address this issue, one can use forecasted wind as an exogenous variable.

We will be running these regressions using the `FixedEffectModels` library.

In [ ]:
reg_w = reg(df, @formula(system_costs ~ wind + fe(year) + fe(month)))

In [ ]:
reg_wf = reg(df, @formula(system_costs ~  wind_forecast + fe(year) + fe(month)))

We might want to instrument wind production with its forecast instead.

In [ ]:
reg_i = reg(df, @formula(system_costs ~  (wind~wind_forecast) + fe(year) + fe(month)))

Another possible problem is that system costs from wind production may be realized in hours with no wind. In this case, the hourly regression coefficient will be downward biased. To circumvent this issue, we can estimate the same regression at a daily level.

For that, we compute the total system costs as well as total wind power.

In [ ]:
df.day_id = string.(df.year,df.month,df.day)
#In Julia, row-wise operations are defined with a dot.
	
df_day = combine(groupby(df, ["day_id","year","month"]), :wind => sum => :wind, :wind_forecast => sum => :wind_forecast, :system_costs => sum => :system_costs);

In [ ]:
reg_d = reg(df_day, @formula(system_costs ~ wind_forecast + fe(year) + fe(month)))

We can display the output of our regressions using the `RegressionTables` package (similar to `stargazer` in R).

In [ ]:
regtable(reg_w, reg_wf, reg_i, reg_d, regression_statistics = [:nobs,:adjr2])

This package also allows you to generate Latex output:

In [ ]:
regtable(reg_w,reg_wf,reg_i,reg_d; renderSettings = latexOutput())

# To create a Latex document with the table simply specify the name of the document:
# regtable(reg_w,reg_wf,reg_i,reg_d; renderSettings = latexOutput("myoutputfile.tex"))


## A policy change 

The paper explores the **role of market design** in affecting the value of wind. The market moved away from subsidies that are paid based on production to subsidies that are based on installed capacity (subject to minimum performance requirments).

In the wholesale market, this implies that renewables no longer have an incentive to produce when prices are very low, e.g., as in California or Texas, in which prices are often zero or negative. 

We will split the data in two to examine the change in the distribution of wholesale prices around the policy change.

In [ ]:
df.policy = (((df.year .> 2014) .| ((df.year.==2014) .& (df.month .> 5))));
df_policy = filter(row -> 2012 < row.year < 2016 ,df);
	
histogram(df_policy.wholesale_price, group = df_policy.policy, alpha = 0.7, label = ["pre" "post"])

The policy change appeared to reduce system costs in the market. This could be due to the challenges of dispatching the market in the presence of zero prices, which lead to several strategic distortions.

We can plot system costs before and after the change.

In [ ]:
binscatter(groupby(df_policy, :policy), @formula(system_costs ~ wind_forecast), 10, 
	seriestype = :linearfit,
	legend = :topleft)

Consumers were still worse off due to the increase in prices. Wind price reduction effect diminished.

In [ ]:
binscatter(groupby(df_policy, :policy), @formula(total_price ~ wind_forecast), 10, 
	seriestype = :linearfit,
	legend = :topright)

**Note:** This is an event study, so there are other changes happening in the market. The idea here is to show major effects of the policy, but proper quantification requires more explicit control of confounders. To start with, although not exhaustive, we can estimate the effect of wind forecast after the policy change.

In [ ]:
reg(df, @formula(wholesale_price ~ wind_forecast*policy + demand_forecast + fe(hour) + fe(year) + fe(month)))

## Follow-up exercises

1. What is the correlation of wind and demand? How could that affect the valuation of wind power?

2. (*) What is the environmental benefit of wind power in this market per unit of wind? Try to quantify that by regressing emissions on wind and converting it to a monetary amount using a valuation for emissions reductions. 
Estimate the total welfare effects of wind production. For that, you need to add to the environmental benefit the consumer and producer surplus. With respect to the producer surplus assume that the LCOE ranges between 50 to 90 €/MWh. How does your answer depend on the monetary value of reducing emissions?